In [5]:
%pip install scikit-learn pandas xgboost  --upgrade

  Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pandas-2.1.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached xgboost-2.0.2-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached numpy-1.26.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.11.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)


In [4]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [ ]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)


In [ ]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [ ]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [ ]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [ ]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [ ]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [ ]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [ ]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [ ]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [ ]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [ ]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [ ]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [ ]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [ ]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [ ]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [ ]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [ ]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [ ]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [ ]:
main_data = pd.read_csv('datasets/unsw_nb_15_cat_in_num_binaryclass_50k_sample.csv')
main_data.head()

,0,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,0,3,3,0.028016,3926,56526,31,29,7,27,...,0.0,0.0,0.0,13,4,2,7,1,1,2
1,0,3,3,1.007838,1580,10168,31,29,3,5,...,1.0,0.0,0.0,2,1,3,4,1,1,1
2,0,3,3,0.037968,2766,27384,31,29,7,16,...,0.0,0.0,0.0,16,6,5,5,1,1,3
3,1,4,2,13.790198,12832,0,254,0,0,0,...,0.0,0.0,0.0,1,1,1,1,1,1,1
4,0,3,3,0.018681,1540,1644,31,29,4,4,...,0.0,0.0,0.0,6,12,12,9,6,1,6


In [ ]:
main_data.columns

Index(['0', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl',
       'sloss', 'dloss', 'service', 'sload', 'dload', 'spkts', 'dpkts', 'swin',
       'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth',
       'res_bdy_len', 'sjit', 'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt',
       'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object')

In [ ]:
main_data.shape

(49999, 44)

In [ ]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

Empty DataFrame
Columns: [0, proto, state, dur, sbytes, dbytes, sttl, dttl, sloss, dloss, service, sload, dload, spkts, dpkts, swin, dwin, stcpb, dtcpb, smeansz, dmeansz, trans_depth, res_bdy_len, sjit, djit, stime, ltime, sintpkt, dintpkt, tcprtt, synack, ackdat, is_sm_ips_ports, ct_state_ttl, ct_flw_http_mthd, is_ftp_login, ct_ftp_cmd, ct_srv_src, ct_srv_dst, ct_dst_ltm, ct_src_ltm, ct_src_dport_ltm, ct_dst_sport_ltm, ct_dst_src_ltm]
Index: []

[0 rows x 44 columns]


""
0
1
2
3
4
...
49994
49995
49996
49997


In [ ]:
class_column = '0' #might need to edit this depending on dataset

In [ ]:
feature_list = main_data
feature_list =  feature_list.drop(class_column, axis=1)
feature_list = feature_list.columns

x = main_data[feature_list]
y = main_data[class_column]

feature_list = main_data.columns

In [ ]:
caim = CAIMD()

x_disc = caim.fit_transform(x, y)

Categorical [14, 20, 31, 34]


# 0  GLOBAL CAIM  19698.96860800065
# 1  GLOBAL CAIM  19346.623547682168
# 2  GLOBAL CAIM  20350.765250883967
# 3  GLOBAL CAIM  22746.154343306458
# 4  GLOBAL CAIM  20326.88745604856
# 5  GLOBAL CAIM  24145.30555174917
# 6  GLOBAL CAIM  19604.953512205066
# 7  GLOBAL CAIM  19375.16483686022
# 8  GLOBAL CAIM  19415.419523265515
# 9  GLOBAL CAIM  19106.946386482137
# 10  GLOBAL CAIM  20425.44994057103
# 11  GLOBAL CAIM  20321.53085044143
# 12  GLOBAL CAIM  19462.903064847582
# 13  GLOBAL CAIM  20322.806889333457
# 15  GLOBAL CAIM  19352.849569774182
# 16  GLOBAL CAIM  19353.217168875715
# 17  GLOBAL CAIM  19352.941897959514
# 18  GLOBAL CAIM  19855.90937076841
# 19  GLOBAL CAIM  20433.670090528576
# 21  GLOBAL CAIM  19095.965062512903
# 22  GLOBAL CAIM  19387.931610106087
# 23  GLOBAL CAIM  19455.566162333733
# 24  GLOBAL CAIM  19293.98303167044
# 25  GLOBAL CAIM  19293.98303167044
# 26  GLOBAL CAIM  19526.08693974442
# 27  GLOBAL CAIM  20132.836018993148
# 28  GLOBAL CAIM  19540.3439336

In [ ]:
data = x_disc
X = x_disc.iloc[:, :-1]
data['class'] = y
print(data.shape)

(49999, 44)


In [ ]:
feature_list = data.columns

In [ ]:
xk_mim = [32, 5, 3, 19, 13, 4, 27, 11, 2, 10, 41, 42, 40, 37, 36, 8, 26, 23, 38, 39, 22, 14, 16, 17, 15, 25, 24, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 34, 18, 31, 12, 33]

In [ ]:
xk_mrmr = [32, 41, 3, 0, 5, 1, 25, 6, 19, 33, 35, 28, 10, 21, 31, 27, 30, 12, 8, 2, 29, 34, 13, 18, 39, 24, 20, 4, 26, 11, 38, 22, 36, 40, 37, 7, 23, 42, 14, 9, 16, 15, 17]

In [ ]:
xk_dimrmr = [32, 9, 3, 0, 5, 25, 6, 19, 1, 33, 41, 28, 31, 10, 35, 21, 27, 30, 12, 2, 29, 34, 13, 4, 18, 8, 11, 24, 20, 39, 42, 26, 40, 36, 37, 38, 22, 23, 7, 14, 16, 15, 17]

In [ ]:
df = pd.read_csv('results/unsw_50ksample_comparison.csv')

# Print the DataFrame
print(df)

# Retrieve data from the DataFrame
mim_vals = df['MIM'].tolist()
mrmr_vals = df['MRMR'].tolist()
dimrmr_vals = df['DIMRMR'].tolist()

[0.0425810236150721, 0.0460153091777011, 0.1989443940673647, 0.2781145333563602, 0.2036743263905853, 0.3198890796854017, 0.038480583482342, 0.0640714568498019, 0.0990195650734251, 0.0564516501022382, 0.1981748810914019, 0.2036328390553388, 0.0001036820919125, 0.2038404059382239, 0.0751407216799058, 0.0745818482301763, 0.0746180482769232, 0.074603566754585, 0.0010134988757222, 0.2191221110209427, 0.0014817867850641, 0.0080965913581536, 0.0815357316684741, 0.0975124150916018, 0.0677514692217602, 0.0677514692217602, 0.0987625162437495, 0.2036535810968995, 0.0370400392900028, 0.0355436741506408, 0.0370432078017789, 0.0003436919347372, 0.4685044452399552, 4.947992971415793e-05, 0.0012537478588277, 0.0012604100362721, 0.1019737997854492, 0.1039715303408643, 0.0887248962622127, 0.0851919518192638, 0.1110086755519712, 0.1430926587409988, 0.1294362041293426] [0.0252638255043953, 0.0064313171486231, -0.0043634664145484, 0.0427845122919468, -0.0129367780854811, 0.0846107609788732, 0.0124742836647

In [ ]:
!python --version

Python 3.8.10


In [55]:
# all features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_dimrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
# print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 0, 5]


KeyError: 'log_loss'

In [40]:
# mrmr top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_mrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 41, 3, 0, 5]
       ct_state_ttl  ct_dst_sport_ltm  sbytes  proto  sttl
0               0.0               0.0     1.0    0.0   0.0
1               0.0               0.0     1.0    0.0   0.0
2               0.0               0.0     1.0    0.0   0.0
3               1.0               0.0     1.0    0.0   1.0
4               0.0               0.0     1.0    0.0   0.0
...             ...               ...     ...    ...   ...
49994           0.0               1.0     1.0    0.0   0.0
49995           0.0               1.0     1.0    0.0   0.0
49996           0.0               0.0     1.0    0.0   0.0
49997           0.0               0.0     1.0    0.0   0.0
49998           0.0               0.0     1.0    0.0   0.0

[49999 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9869


In [41]:
# dimrmr top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_dimrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 0, 5]
       ct_state_ttl  service  sbytes  proto  sttl
0               0.0      1.0     1.0    0.0   0.0
1               0.0      1.0     1.0    0.0   0.0
2               0.0      1.0     1.0    0.0   0.0
3               1.0      1.0     1.0    0.0   1.0
4               0.0      1.0     1.0    0.0   0.0
...             ...      ...     ...    ...   ...
49994           0.0      1.0     1.0    0.0   0.0
49995           0.0      1.0     1.0    0.0   0.0
49996           0.0      1.0     1.0    0.0   0.0
49997           0.0      1.0     1.0    0.0   0.0
49998           0.0      1.0     1.0    0.0   0.0

[49999 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9874


In [42]:
print(xk_mim)
print(xk_mrmr)
print(xk_dimrmr)

[32, 5, 3, 19, 13, 4, 27, 11, 2, 10, 41, 42, 40, 37, 36, 8, 26, 23, 38, 39, 22, 14, 16, 17, 15, 25, 24, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 34, 18, 31, 12, 33]
[32, 41, 3, 0, 5, 1, 25, 6, 19, 33, 35, 28, 10, 21, 31, 27, 30, 12, 8, 2, 29, 34, 13, 18, 39, 24, 20, 4, 26, 11, 38, 22, 36, 40, 37, 7, 23, 42, 14, 9, 16, 15, 17]
[32, 9, 3, 0, 5, 25, 6, 19, 1, 33, 41, 28, 31, 10, 35, 21, 27, 30, 12, 2, 29, 34, 13, 4, 18, 8, 11, 24, 20, 39, 42, 26, 40, 36, 37, 38, 22, 23, 7, 14, 16, 15, 17]


In [46]:
# improvized five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = [32, 9, 3, 0]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 0]
       ct_state_ttl  service  sbytes  proto
0               0.0      1.0     1.0    0.0
1               0.0      1.0     1.0    0.0
2               0.0      1.0     1.0    0.0
3               1.0      1.0     1.0    0.0
4               0.0      1.0     1.0    0.0
...             ...      ...     ...    ...
49994           0.0      1.0     1.0    0.0
49995           0.0      1.0     1.0    0.0
49996           0.0      1.0     1.0    0.0
49997           0.0      1.0     1.0    0.0
49998           0.0      1.0     1.0    0.0

[49999 rows x 4 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9874


In [47]:
# improvized five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = [32, 9, 3, 30]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 30]
       ct_state_ttl  service  sbytes  ackdat
0               0.0      1.0     1.0     0.0
1               0.0      1.0     1.0     0.0
2               0.0      1.0     1.0     0.0
3               1.0      1.0     1.0     0.0
4               0.0      1.0     1.0     0.0
...             ...      ...     ...     ...
49994           0.0      1.0     1.0     0.0
49995           0.0      1.0     1.0     0.0
49996           0.0      1.0     1.0     0.0
49997           0.0      1.0     1.0     0.0
49998           0.0      1.0     1.0     0.0

[49999 rows x 4 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9867
